In [1]:
!pip install pyensae

In [2]:
!pip install qgrid

In [1]:
%load_ext pyensae

In [3]:
%SQL_connect ./databases/baseball_full.db

### **Question 1, Part a**
Calculating the ERAs considered OK for Minnesota Twins and Milwauke Brewers Players.

**dplyr code**

`Pitching
    %>% select(playerID, teamID, ER, IPouts)
    %>% filter((teamID == "MIL" | teamID == "MIN") & YearID == '2000)
    %>% mutate(Calc_Era = (ER/(Ipouts/3))*9
    %>% filter(Calc_ERA < 4)
    %>% select(playerID, teamID, Calc_ERA)`

In [27]:
%%SQL
    SELECT PlayerID, TeamID, (ER/(IPouts/3.0))*9 AS Calc_ERA
    FROM Pitching
    WHERE  TeamID IN("MIL", "MIN") AND YearID = '2000' AND Calc_ERA < 4

,playerID,teamID,Calc_ERA,ERA
0,aceveju01,MIL,3.810484,3.81
1,damicje01,MIL,2.661191,2.66
2,guarded01,MIN,3.940541,3.94
3,hawkila01,MIN,3.387833,3.39
4,kingra01,MIL,1.255814,1.26
5,leskacu01,MIL,2.560345,2.56
6,milletr01,MIN,3.895522,3.90
7,weathda01,MIL,3.065502,3.07
8,wellsbo01,MIN,3.648649,3.65
9,wickmbo01,MIL,2.934783,2.93


### Question 1, Part b

Calculating the ERA for the Minnesota Twins and Milwauke Brewers for the years 2000, 2005, 2010, and 2015.

**dplyr code**

`Teams
    %>% select(teamID, yearID, ER, IPouts)
    %>% filter((teamID == "MIL" | teamID == "MIN) & (yearID %in% ('2000', '2005', '2010', '2015)))
    %>% mutate(Calc_ERA = (ER/(IPouts/3))*9)
    %>% select(yearID, teamID, Calc_ERA)
    %>% arrange(yearID, teamID)`

In [30]:
%%SQL
    SELECT teamID, yearID, (ER/(IPouts/3.0))*9 AS Calc_ERA
    FROM Teams
    WHERE  teamID IN("MIL", "MIN") AND yearID IN ('2000', '2005', '2010', '2015')

,teamID,yearID,Calc_ERA
0,MIL,2000,4.634008
1,MIN,2000,5.144951
2,MIL,2005,3.974270
3,MIN,2005,3.712270
4,MIL,2010,4.584434
5,MIN,2010,3.952731
6,MIL,2015,4.277352
7,MIN,2015,4.072765


### Question 2, Part a

Finding the average salary by team for 2006 and 2012.  

**dplyr code**

`(Franchises
    %>% select(franchID, franchName)
  ) -> Franch
  
  (Teams
      %>% select(yearID, teamID, franchID)
      %>% left_join(Franch, by = "franchID")
      %>% select(teamID, franchName)
  )-> Teams_Franch
  
  Salaries
      %>% select(yearID, teamID, salary)
      %>% filter(yearID %in% ('2006', '2012')
      %>% inner_join(Teams_Franch, by = "teamID")
      %>% group_by(franchName, teamID, yearID)
      %>% summarize(Avg_Salary = mean(salary))`

In [36]:
%%SQL
    SELECT YearID, franchName, teamID, AVG(Salary) AS Avg_Salary
    FROM (SELECT A.yearID, A.teamID, A.Salary, B.teamID, B.franchName
             FROM Salaries AS A
                 LEFT JOIN(SELECT A.yearID, A.teamID, A.franchID, B.franchID, B.franchName
                              FROM Teams AS A
                                  LEFT JOIN TeamsFranchises AS B
                                ON A.franchID = B.franchID) AS B
              ON A.teamID = B.teamID)
    WHERE yearID IN ('2006', '2012')
    GROUP BY franchName, teamID, yearID

,yearID,franchName,teamID,Avg_Salary
0,2006,Arizona Diamondbacks,ARI,2.295547e+06
1,2012,Arizona Diamondbacks,ARI,2.733512e+06
2,2006,Atlanta Braves,ATL,3.108858e+06
3,2012,Atlanta Braves,ATL,2.856205e+06
4,2006,Baltimore Orioles,BAL,2.592342e+06
5,2012,Baltimore Orioles,BAL,2.762643e+06
6,2006,Boston Red Sox,BOS,4.448142e+06
7,2012,Boston Red Sox,BOS,5.093724e+06
8,2006,Chicago Cubs,CHN,3.372304e+06
9,2012,Chicago Cubs,CHN,3.392194e+06


### Question 2, Part b

Find the team with the largest salary overall, regardless of year.

**dplyr code**

`Salaries
    %>% select(yearID, teamID, salary)
    %>% inner_join(Teams_Franch, by = "teamID")
    %>% group_by(franchName, teamID, yearID)
    %>% summarize(Avg_Salary = mean(salary))
    arrange(desc(Avg_Salary))`

In [40]:
%%SQL
    SELECT YearID, franchName, teamID, AVG(Salary) AS Avg_Salary
    FROM (SELECT A.yearID, A.teamID, A.Salary, B.teamID, B.franchName
             FROM Salaries AS A
                 LEFT JOIN(SELECT A.yearID, A.teamID, A.franchID, B.franchID, B.franchName
                              FROM Teams AS A
                                  LEFT JOIN TeamsFranchises AS B
                                ON A.franchID = B.franchID) AS B
              ON A.teamID = B.teamID)
    GROUP BY franchName, teamID, yearID
    ORDER BY Avg_Salary DESC

,yearID,franchName,teamID,Avg_Salary
0,2010,New York Yankees,NYA,8.253336e+06
1,2014,New York Yankees,NYA,8.230996e+06
2,2005,New York Yankees,NYA,8.011801e+06
3,2009,New York Yankees,NYA,7.748046e+06
4,2016,New York Yankees,NYA,7.689579e+06
5,2013,New York Yankees,NYA,7.483190e+06
6,2015,Los Angeles Dodgers,LAN,7.441103e+06
7,2015,New York Yankees,NYA,7.336274e+06
8,2013,Los Angeles Dodgers,LAN,6.980069e+06
9,2011,New York Yankees,NYA,6.975001e+06


### Question 3

Get a list of the players' names for the Detriot Tigers that have been born since 1980.

**dplyr code**

`(Batting
    %>% select(teamID, playerID)
    %>% distinct(teamID, playerID)
    %>% filter(teamID == "DET")
  ) -> Batting2
  
  People
      %>% select(nameFirst, nameLast, playerID, birthYear)
      %>% filter(birthYear >= "1980")
      %>% left_join(Batting2, by = "playerID")
      %>% filter(teamID != "<NA>")
      %>% arrange(desc(nameLast))
      %>% transmute(TeamID = teamID, Player_Name = paste(namefirst, nameLast, sep = " "))`

In [44]:
%%SQL
    SELECT teamID, CONCAT(nameFirst, nameLast) AS Player_Name
    FROM (SELECT A.nameFirst, A.nameLast, A.playerID, A.birthYear, B.teamID, B.playerID
             FROM (SELECT nameFirst, nameLast, playerID, birthYear
                      FROM People WHERE birthYear >=190) AS A
                         INNER JOIN (SELECT teamID, playerID 
                                        FROM Batting WHERE teamID == "DET") AS B
         ON A.playerID = B.playerID)
    ORDER BY A.nameLast DESC

unable to execute a SQL request (1)(file ./databases/baseball_full.db)
SELECT teamID, CONCAT(nameFirst, nameLast) AS Player_Name
    FROM (SELECT A.nameFirst, A.nameLast, A.playerID, A.birthYear, B.teamID, B.playerID
             FROM (SELECT nameFirst, nameLast, playerID, birthYear
                      FROM People WHERE birthYear >=190) AS A
                         INNER JOIN (SELECT teamID, playerID 
                                        FROM Batting WHERE teamID == "DET") AS B
         ON A.playerID = B.playerID)
    ORDER BY A.nameLast DESC
no such function: CONCAT
'SELECT teamID, CONCAT(nameFirst, nameLast) AS Player_Name
    FROM (SELECT A.nameFirst, A.nameLast, A.playerID, A.birthYear, B.teamID, B.playerID
             FROM (SELECT nameFirst, nameLast, playerID, birthYear
                      FROM People WHERE birthYear >=190) AS A
                         INNER JOIN (SELECT teamID, playerID 
                                        FROM Batting WHERE teamID == "DET") AS B
 

### Question 4

Calculating OPS for each player since 2010.

**dplyr code**

`Batting
    %>% filter(yearID >= "2010" & HBP != "NA" & SH != "NA")
    %>% transmute(ID = playerID, OBP = ((H + BB + HBP)/(AB + BB + SH + HBP)), SLG = ((H + 2*X2B + 3*X3B + 4*HR)/AB))
    %>% transmute(ID = ID, OBP = OBP, SLG = SLG, OPS = OBP + SLG)
    %>% filter(OPS != "NaN" & OPS != "0")`

In [47]:
%%SQL
    SELECT playerID, ((H + BB + HBP) /(AB + BB + SH + HBP)) AS OBP, ((H + 2*X2B + 3*X3B + 4*HR)/AB) AS SLG,
    (OBP + SLG) AS OPS
    FROM Batting
    WHERE yearID >= 2010 AND OPS != "." AND OPS > 0

unable to execute a SQL request (1)(file ./databases/baseball_full.db)
SELECT playerID, ((H + BB + HBP) /(AB + BB + SH + HBP)) AS OBP, ((H + 2*X2B + 3*X3B + 4*HR)/AB) AS SLG,
    (OBP + SLG) AS OPS
    FROM Batting
    WHERE yearID >= 2010 AND OPS != "." AND OPS > 0
no such column: X2B
'SELECT playerID, ((H + BB + HBP) /(AB + BB + SH + HBP)) AS OBP, ((H + 2*X2B + 3*X3B + 4*HR)/AB) AS SLG,
    (OBP + SLG) AS OPS
    FROM Batting
    WHERE yearID >= 2010 AND OPS != "." AND OPS > 0'


### Question 5

Find the number of players from each college or university for 2009.

**dplyr code**

`(CollegePlaying
    %>% select(schooID, yearID)
    %>% filter(yearID == '2009)
    %>% group_by(schoolID)
    %>% summarize(Num_Play = n())
  ) -> College
  
  Schools
      %>% select(name_full, schoolID)
      %>% left_join(College, by = "schoolID")
      %>% arrange(desc(Num_Play))
      %>% select(name_full, Num_Play)`

In [22]:
%%SQL
    SELECT A.name_Full, B.Num_Play
    FROM ( 
    (SELECT name_Full, SchoolID FROM Schools) AS A
        INNER JOIN 
            (SELECT SchoolID, COUNT(SchoolID) AS Num_Play, YearID FROM CollegePlaying
                WHERE YearID = 2009 
                GROUP BY SchoolID) AS B
        ON A.SchoolID = B.SchoolID
        )
    ORDER BY Num_Play DESC

,name_Full,Num_Play
0,University of North Carolina at Chapel Hill,5
1,Arizona State University,4
2,"University of California, Berkeley",4
3,University of Texas at Austin,4
4,Tulane University,4
5,Dallas Baptist University,3
6,Nova Southeastern University,3
7,University of Florida,3
8,University of Miami,3
9,University of South Carolina,3
